# Login and Initialize
## Login Using Spotify
After running this click Sign-In and login to Spotify. **Note** you cannot run the whole Notebook through. Run cell #1 first.

In [ ]:
%load_ext autoreload
%autoreload 2
from ipyauth import ParamsSpotify, Auth
auth = Auth(ParamsSpotify(
    redirect_uri='http://localhost:8888/callback', 
    client_id="9e4657eefbac41afa98c61f590d8fd51"))
auth

## Fetch the Library
Initialize the Spotify object with a `logger` and fetch the library DataFrame. Check it works by outputting the Profile image.

In [ ]:
import requests
from IPython.display import Image
from spotify import Spotify
from pandas import DataFrame as df
from datetime import datetime,timedelta,date
import json
import logging

logging.basicConfig()
defaultLogger = logging.getLogger()
defaultLogger.setLevel(logging.ERROR)
logger = logging.getLogger('SpotifyLab')
logger.setLevel(logging.DEBUG)

spot = Spotify(auth, logger = logger)
lib = spot.fetchLibraryDataFrame( cache=False)
#lib.columns.values
Image(url=spot.imageUrl, width=50)

In [ ]:
#lib.columns.values
#import pandas as pd
decSongs = spot.fetchPlaylistUris("December Songs")
holidaySongs = spot.fetchPlaylistUris("Holiday Songs")
allDecember = lib[(lib.track_uri.isin(decSongs)) | (lib.track_uri.isin(holidaySongs)) ]
spot.updatePlaylist("Auto December",allDecember)
spot.updatePlaylist("Auto Holly Jolly", allDecember[ allDecember.energy>0.7 ])
spot.updatePlaylist("Auto December Chill", allDecember[ allDecember.energy<0.701 ])
spot.updatePlaylist("Auto Silent Night", allDecember[ allDecember.energy<0.3 ])
spot.updatePlaylist("Auto Instrumental Christmas", allDecember[ allDecember.instrumentalness>0.75 ])
#spot.fetchPlaylistUris("December Songs")

#lib[lib.track_name=="I Want A Dog"][["track_name","artist","genres","track_released","release_date","name","track_album_name","track_album_id","track_album_release_date","track_album_release_date_precision"]]

# Playlist Updates

## A Running Playlist

This playlist has high tempo and energy. The preference is for instrumentals with high danceability. Avoid long songs to keep the tracks changing.

In [ ]:
newPlaylist = lib[
    (lib.tempo>130) & 
    (lib.instrumentalness>0.3) & 
    (lib.duration_ms<1000*60*6) & 
    (lib.danceability>0.7) &
    (lib.track_released>datetime(1990,1,1))
].sort_values(["added_at"])
# Exclude some artists from this playlist
newPlaylist = newPlaylist[~newPlaylist.artist.isin(["Radiohead","Rodrigo y Gabriela","2CELLOS","Faithless","Doves"])]
spot.updatePlaylist("Auto Run Fast",newPlaylist,excludingPlaylists=["Holiday Songs"])

## Unpopular or Hispter?
This playlists tracks tracks with low popularity. This is useful for checking if you added the most commonly played track (e.g. a version before it was remastered).

In [ ]:
spot.updatePlaylist("Are You Sure",lib[lib.track_popularity<2],excludingPlaylists=["Holiday Songs"])

## Decades of Music
These playlists bucket the music in your library into decades using the release date. 

In [ ]:
spot.updatePlaylist("auto 1960s",lib[lib.track_released<datetime(1970,1,1)],excludingPlaylists=["Holiday Songs"])
spot.updatePlaylist("auto 1970s",lib[(lib.track_released>datetime(1970,1,1)) & (lib.track_released<datetime(1980,1,1))],excludingPlaylists=["Holiday Songs"])
spot.updatePlaylist("auto 1980s",lib[(lib.track_released>datetime(1980,1,1)) & (lib.track_released<datetime(1990,1,1))],excludingPlaylists=["Holiday Songs"])
spot.updatePlaylist("auto 1990s",lib[(lib.track_released>datetime(1990,1,1)) & (lib.track_released<datetime(2000,1,1))],excludingPlaylists=["Holiday Songs"])
spot.updatePlaylist("auto 2000s",lib[(lib.track_released>datetime(2000,1,1)) & (lib.track_released<datetime(2010,1,1))],excludingPlaylists=["Holiday Songs"])
spot.updatePlaylist("auto 2010s",lib[(lib.track_released>datetime(2010,1,1)) & (lib.track_released<datetime(2020,1,1))],excludingPlaylists=["Holiday Songs"])
spot.updatePlaylist("2019 So Far",lib[lib.track_released>datetime(2019,1,1)],excludingPlaylists=["Holiday Songs"])


## Recently Added
The music you added in the last couple of months.

In [ ]:
spot.updatePlaylist("Auto Two Weeks",lib[lib.added_at>(datetime.now()-timedelta(days=14)).isoformat()],excludingPlaylists=["Holiday Songs"])
spot.updatePlaylist("Auto Recent Added",lib[lib.added_at>(datetime.now()-timedelta(days=60)).isoformat()],excludingPlaylists=["Holiday Songs"])
spot.updatePlaylist("Auto Recent",lib[(lib.track_released>(datetime.now()-timedelta(days=90)))],excludingPlaylists=["Holiday Songs"])
spot.updatePlaylist("New Mix Takes",lib[(lib.track_released>(datetime.now()-timedelta(days=90)))
                                        & (lib.energy>0.3)],excludingPlaylists=["Holiday Songs"])

## Mood Music
Music for every occasion. Low energy dinner parties, going to sleep or concentrating at work.

In [ ]:
spot.updatePlaylist("Dinner Party",lib[(lib.acousticness>0.9) & (lib.energy<0.6)],excludingPlaylists=["Holiday Songs"])
spot.updatePlaylist("Night Nights",lib[(lib.acousticness>0.9) & (lib.energy<0.3) & (lib.danceability<0.3)],excludingPlaylists=["Holiday Songs"])
spot.updatePlaylist("To Work To",lib[(lib.instrumentalness>0.9) & (lib.energy>0.6) ],excludingPlaylists=["Holiday Songs"])
spot.updatePlaylist("Quiet Work",lib[(lib.instrumentalness>0.9) & (lib.danceability<0.6) & (lib.energy>0.3) ],excludingPlaylists=["Holiday Songs"])
# With new Music Only
spot.updatePlaylist("Recent Work Music",lib[
    (lib.instrumentalness>0.9) & 
    (lib.energy>0.6) & 
    (lib.added_at>(datetime.now()-timedelta(days=90)).isoformat() )],excludingPlaylists=["Holiday Songs"])
spot.updatePlaylist("Recent Quiet Work",lib[
    (lib.instrumentalness>0.9) & 
    (lib.danceability<0.6) & 
    (lib.energy>0.3) &
    (lib.added_at>(datetime.now()-timedelta(days=90)).isoformat()) ],excludingPlaylists=["Holiday Songs"])

## More Analysis
It's easy to add the same track twice, from different albums. This can cause duplicates in playlists. We can use Pandas `groupby` to scan for that

In [ ]:
# Duplicates in the library
ff = lib.groupby(["track_name","artist"]).count()
(ff[ff.track_uri>1]).index.values